In [74]:
pip install api

  Created wheel for api: filename=api-0.0.7-py3-none-any.whl size=2325 sha256=53355d1d4f606ac0c2ec4f08fd3a2c9ca82044ff90ad819831d5cc9152d9b791
  Stored in directory: c:\users\ramse\appdata\local\pip\cache\wheels\f9\aa\62\7a58dc8d2d5b68a3b46506e92d896a7128456adb95489ba973
Successfully built api
Note: you may need to restart the kernel to use updated packages.


In [104]:

import requests
import json

# Set the Salesforce instance URL, OAuth client ID, and client secret

salesforce_url = "https://login.salesforce.com/services/oauth2/token"
client_id = "3MVG9ux34Ig8G5epaMPqbA1E25OpLuKGuGcWZixMzgV6myFvvKoIQnGrMY5mg9pTNHPBWj9GgJNuwD0TAIEIy"
client_secret = "50D529BA45FA479E8FE492C5BC5CBE774452FF21597B1ED8D73352105428F3FC"
username = "sfdcshared@gmail.com"
password = "G6CaYJ0riOQz7Oes2yH9RS7n"

# Set the REST API endpoint URL for obtaining an access token
rest_api_url = salesforce_url+"/services/oauth2/token"

payload = {
    "grant_type":"password",
    "client_id": client_id,
    "client_secret":client_secret,
    "username": username,
    "password": password
}

# Send the POST request to the REST API endpoint with the payload
response = requests.post(salesforce_url, data=payload)
print('Done')
print(response.status_code)
print(response.reason)

response.json()




Done
200
OK


{'access_token': '00DDn000002q7Dy!ARkAQPw.b4iqmNOXGXCrP.Pkm3gqHzJXJensAXFWPiMDqP7DPWbxeUYgaayD2sgzIzYzI6e1_Z0OUujGTaHm_v1V8LdaFRRY',
 'instance_url': 'https://koresoftware-dev-ed.develop.my.salesforce.com',
 'id': 'https://login.salesforce.com/id/00DDn000002q7DyMAI/005Dn000002M1PFIA0',
 'token_type': 'Bearer',
 'issued_at': '1678412198019',
 'signature': 'mrVr+B6V5S5cMaZE6JaHVKsICkFs82GwXrqcOvzIrZ8='}

In [105]:
access_token = response.json().get("access_token")
instance_url = response.json().get("instance_url")

In [135]:
# Set the headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Define the SOQL query
#query = "Select IndividualId,Email,Suffix,FirstName,LastName,MailingStreet,MailingCity,MailingState,MailingPostalCode,DoNotCall,Phone FROM Contact"
query = "Select IndividualId,OwnerId,Email,FirstName,LastName,MailingStreet,MailingCity,MailingState,MailingPostalCode,MailingCountry,DoNotCall,Phone,MobilePhone from Contact where DoNotCall<>True"
# Construct the URL
url = f"{instance_url}/services/data/v51.0/query?q={query}"

# Make the request
response = requests.get(url, headers=headers)

# Process the response
if response.status_code == 200:
    data = response.json()
    with open("output_notTrue.json", "w") as f:
        json.dump(data, f)
else:
    print(f"Error retrieving data: {response.status_code}")
    
